In [16]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import pandas as pd 


**Brief description of the problem and data (5 pts)**

Briefly describe the challenge problem and NLP. Describe the size, dimension, structure, etc., of the data.

This competition is based around Generative Adversarial Networks. In essence, in this instance, you create two networks. one that can tell the difference  between a real image, and a fake. And another that creates fake images. In this case there is a twist, and that is it will take an image, and try and create the same basic image, but as a monet painting. The basic data structure contains four folders, two of them, sets of image in JPEG format (monet(300), and photos(7038)), the second are the same images in TfRec format (guessing its a tensorflow ready file format). There are no sample output files, the idea is that when the model is finished it will output between 7-10K monet style (at least thransform the 7038 files in photos to their monet equivalents, add more if desired).

In [17]:
# Paths to simplify navigation. 
DATA = '/kaggle/input/gan-getting-started/'
MONET_JPG = os.path.join(DATA, 'monet_jpg')
PHOTO_JPG = os.path.join(DATA, 'photo_jpg')
MONET_TFREC = os.path.join(DATA, 'monet_tfrec')
PHOTO_TFREC = os.path.join(DATA, 'photo_tfrec')

First we will load the photos normally so we can analyse them. 

In [18]:
'''monet_files = sorted(glob.glob(os.path.join(MONET_JPG, '*.jpg')))
photo_files = sorted(glob.glob(os.path.join(PHOTO_JPG, '*.jpg')))

print(f"Number of Monet images: {len(monet_files)}")
print(f"Number of Photo images: {len(photo_files)}")

# Load and display a sample Monet image
sample_monet = load_img(monet_files[0])
plt.imshow(sample_monet)
plt.title("Sample Monet Painting")
plt.axis('off')
plt.show()

# Similarly for a photo
sample_photo = load_img(photo_files[0])
plt.imshow(sample_photo)
plt.title("Sample Photo")
plt.axis('off')
plt.show()'''

'monet_files = sorted(glob.glob(os.path.join(MONET_JPG, \'*.jpg\')))\nphoto_files = sorted(glob.glob(os.path.join(PHOTO_JPG, \'*.jpg\')))\n\nprint(f"Number of Monet images: {len(monet_files)}")\nprint(f"Number of Photo images: {len(photo_files)}")\n\n# Load and display a sample Monet image\nsample_monet = load_img(monet_files[0])\nplt.imshow(sample_monet)\nplt.title("Sample Monet Painting")\nplt.axis(\'off\')\nplt.show()\n\n# Similarly for a photo\nsample_photo = load_img(photo_files[0])\nplt.imshow(sample_photo)\nplt.title("Sample Photo")\nplt.axis(\'off\')\nplt.show()'

We are going to attempt to use the TFRecs in the model pipeline in hopes of faster training as I am very close to out of GPU time on my account. So here goes nothing!

In [19]:
'''# Parsing function (from earlier)
def parse_tfrecord(example):
    features = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'image_name': tf.io.FixedLenFeature([], tf.string)
    }
    parsed = tf.io.parse_single_example(example, features)
    image = tf.io.decode_jpeg(parsed['image'], channels=3)
    image = tf.image.resize(image, [256, 256])
    image = (tf.cast(image, tf.float32) / 127.5) - 1.0  # [-1,1] norm for GAN stability
    return image

# Augmentation function (simple for now)
def augment(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image

# Build datasets
BATCH_SIZE = 1  # CycleGAN often uses 1 for instance norm; increase if hardware allows
BUFFER_SIZE = 300  # For shuffling Monets (small set)

monet_tfrec_files = sorted(glob.glob(os.path.join(MONET_TFREC, '*.tfrec')))
photo_tfrec_files = sorted(glob.glob(os.path.join(PHOTO_TFREC, '*.tfrec')))

monet_ds = tf.data.TFRecordDataset(monet_tfrec_files)
monet_ds = monet_ds.map(parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)
monet_ds = monet_ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
monet_ds = monet_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

photo_ds = tf.data.TFRecordDataset(photo_tfrec_files)
photo_ds = photo_ds.map(parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)
photo_ds = photo_ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
photo_ds = photo_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)  # Larger shuffle for photos

# Zip in single dataset
train_ds = tf.data.Dataset.zip((monet_ds, photo_ds))'''

"# Parsing function (from earlier)\ndef parse_tfrecord(example):\n    features = {\n        'image': tf.io.FixedLenFeature([], tf.string),\n        'image_name': tf.io.FixedLenFeature([], tf.string)\n    }\n    parsed = tf.io.parse_single_example(example, features)\n    image = tf.io.decode_jpeg(parsed['image'], channels=3)\n    image = tf.image.resize(image, [256, 256])\n    image = (tf.cast(image, tf.float32) / 127.5) - 1.0  # [-1,1] norm for GAN stability\n    return image\n\n# Augmentation function (simple for now)\ndef augment(image):\n    image = tf.image.random_flip_left_right(image)\n    image = tf.image.random_flip_up_down(image)\n    return image\n\n# Build datasets\nBATCH_SIZE = 1  # CycleGAN often uses 1 for instance norm; increase if hardware allows\nBUFFER_SIZE = 300  # For shuffling Monets (small set)\n\nmonet_tfrec_files = sorted(glob.glob(os.path.join(MONET_TFREC, '*.tfrec')))\nphoto_tfrec_files = sorted(glob.glob(os.path.join(PHOTO_TFREC, '*.tfrec')))\n\nmonet_ds = tf

In [20]:
'''for monet, photo in train_ds.take(1):
    # Visualize Monet sample (denormalize from [-1,1] to [0,1] for display)
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.imshow((monet[0] + 1) / 2)
    plt.title("Sample Monet")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow((photo[0] + 1) / 2)
    plt.title("Sample Photo")
    plt.axis('off')
    
    plt.show()'''

'for monet, photo in train_ds.take(1):\n    # Visualize Monet sample (denormalize from [-1,1] to [0,1] for display)\n    plt.figure(figsize=(10, 5))\n    \n    plt.subplot(1, 2, 1)\n    plt.imshow((monet[0] + 1) / 2)\n    plt.title("Sample Monet")\n    plt.axis(\'off\')\n    \n    plt.subplot(1, 2, 2)\n    plt.imshow((photo[0] + 1) / 2)\n    plt.title("Sample Photo")\n    plt.axis(\'off\')\n    \n    plt.show()'

In [21]:
'''# Verify shapes and norms
print("Monet shape:", monet.shape)  # Should be (1, 256, 256, 3)
print("Photo shape:", photo.shape)  # Should be (1, 256, 256, 3)
print("Monet min/max:", tf.reduce_min(monet).numpy(), tf.reduce_max(monet).numpy())  # Expect ~ -1 to 1
print("Photo min/max:", tf.reduce_min(photo).numpy(), tf.reduce_max(photo).numpy())  # Expect ~ -1 to 1'''

'# Verify shapes and norms\nprint("Monet shape:", monet.shape)  # Should be (1, 256, 256, 3)\nprint("Photo shape:", photo.shape)  # Should be (1, 256, 256, 3)\nprint("Monet min/max:", tf.reduce_min(monet).numpy(), tf.reduce_max(monet).numpy())  # Expect ~ -1 to 1\nprint("Photo min/max:", tf.reduce_min(photo).numpy(), tf.reduce_max(photo).numpy())  # Expect ~ -1 to 1'

TFRec data has been imported, and the train dataset file apears to be working correctly. We will move onto EDA and see if there is anything we can find to analyse the picture data given.

**Exploratory Data Analysis (EDA) — Inspect, Visualize and Clean the Data (15 pts)**

Show a few visualizations like histograms. Describe any data cleaning procedures. Based on your EDA, what is your plan of analysis? 

Next we will inspect the data, check for invalid files, verify picture size is all uniform, visualize a few pictures from each, and maybe plot a few histograms.

In [22]:
'''def check_images(files):
    invalid = []
    for f in files:
        try:
            img = Image.open(f)
            img.verify()
            img.close()
        except (IOError, SyntaxError):
            invalid.append(f)
    return invalid

invalid_monet = check_images(monet_files)
invalid_photo = check_images(photo_files)
print(f"Invalid Monet: {invalid_monet}")
print(f"Invalid Photos: {invalid_photo}")'''

'def check_images(files):\n    invalid = []\n    for f in files:\n        try:\n            img = Image.open(f)\n            img.verify()\n            img.close()\n        except (IOError, SyntaxError):\n            invalid.append(f)\n    return invalid\n\ninvalid_monet = check_images(monet_files)\ninvalid_photo = check_images(photo_files)\nprint(f"Invalid Monet: {invalid_monet}")\nprint(f"Invalid Photos: {invalid_photo}")'

Next we will verify picture dimensions.

In [23]:
'''def verify_dimensions(files):
    for f in files:
        img = load_img(f)
        arr = img_to_array(img)
        if arr.shape != (256, 256, 3):
            print(f"Invalid shape for {f}: {arr.shape}")

verify_dimensions(monet_files[:len(monet_files)])'''

'def verify_dimensions(files):\n    for f in files:\n        img = load_img(f)\n        arr = img_to_array(img)\n        if arr.shape != (256, 256, 3):\n            print(f"Invalid shape for {f}: {arr.shape}")\n\nverify_dimensions(monet_files[:len(monet_files)])'

No problems there, all files (256, 256, 3). Next we will plot out a grid of pictures from each set to get a better idea whnat tghe datasets consist of.

In [24]:
'''def plot_grid(files, title, n=25):
    plt.figure(figsize=(10, 10))
    for i, f in enumerate(files[:n]):
        plt.subplot(5, 5, i+1)
        img = load_img(f)
        plt.imshow(img)
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

plot_grid(monet_files, "Monet Paintings Grid")
plot_grid(photo_files, "Photos Grid")'''

'def plot_grid(files, title, n=25):\n    plt.figure(figsize=(10, 10))\n    for i, f in enumerate(files[:n]):\n        plt.subplot(5, 5, i+1)\n        img = load_img(f)\n        plt.imshow(img)\n        plt.axis(\'off\')\n    plt.suptitle(title)\n    plt.show()\n\nplot_grid(monet_files, "Monet Paintings Grid")\nplot_grid(photo_files, "Photos Grid")'

And finally we will plot a color histogram to see if there is anything that stands out about the monet paintings vs the real photos.

In [25]:
'''def plot_histogram(files, title, color='rgb', n=50, clip_max=22000):
    images = np.array([img_to_array(load_img(f)) for f in files[:n]])
    plt.figure()
    for i, c in enumerate(color):
        hist = np.histogram(images[..., i].flatten(), bins=256)[0]
        if clip_max is not None:
            hist = np.clip(hist, 0, clip_max) 
        plt.plot(hist, color=c)
    plt.title(title)
    if clip_max is not None:
        plt.ylim(0, clip_max)
    plt.show()

plot_histogram(monet_files, "Monet Histogram")
plot_histogram(photo_files, "Photo Histogram")'''

'def plot_histogram(files, title, color=\'rgb\', n=50, clip_max=22000):\n    images = np.array([img_to_array(load_img(f)) for f in files[:n]])\n    plt.figure()\n    for i, c in enumerate(color):\n        hist = np.histogram(images[..., i].flatten(), bins=256)[0]\n        if clip_max is not None:\n            hist = np.clip(hist, 0, clip_max) \n        plt.plot(hist, color=c)\n    plt.title(title)\n    if clip_max is not None:\n        plt.ylim(0, clip_max)\n    plt.show()\n\nplot_histogram(monet_files, "Monet Histogram")\nplot_histogram(photo_files, "Photo Histogram")'

One think I find interesting from these histograms is very few blacks or bright colors in the monets, and plenty of blacks in the photos. 

**DModel Architecture (25 pts)**

Describe your model architecture and reasoning for why you believe that specific architecture would be suitable for this problem. Compare multiple architectures and tune hyperparameters. 

My model will consist of a CycleGan: two generators(photo -> monet, Monet -> photo) and two discriminators using the Tensorflow CycleGan tutorial for reference

In [26]:
'''
from tensorflow.keras import layers

def downsample(filters, size, apply_norm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(layers.Conv2D(filters, size, strides=2, padding='same', kernel_initializer=initializer, use_bias=False))
    if apply_norm:
        result.add(layers.LayerNormalization())  
    result.add(layers.LeakyReLU())
    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(layers.Conv2DTranspose(filters, size, strides=2, padding='same', kernel_initializer=initializer, use_bias=False))
    result.add(layers.LayerNormalization())
    if apply_dropout:
        result.add(layers.Dropout(0.5))
    result.add(layers.ReLU())
    return result

# Generator: U-Net like
def Generator():
    inputs = layers.Input(shape=[256, 256, 3])
    
    down_stack = [
        downsample(64, 4, apply_norm=False),  # (bs, 128, 128, 64)
        downsample(128, 4),  # (bs, 64, 64, 128)
        downsample(256, 4),  # (bs, 32, 32, 256)
        downsample(512, 4),  # (bs, 16, 16, 512)
        downsample(512, 4),  # (bs, 8, 8, 512)
        downsample(512, 4),  # (bs, 4, 4, 512)
        downsample(512, 4),  # (bs, 2, 2, 512)
        downsample(512, 4),  # (bs, 1, 1, 512)
    ]
    
    up_stack = [
        upsample(512, 4, apply_dropout=True),  # (bs, 2, 2, 1024)
        upsample(512, 4, apply_dropout=True),  # (bs, 4, 4, 1024)
        upsample(512, 4, apply_dropout=True),  # (bs, 8, 8, 1024)
        upsample(512, 4),  # (bs, 16, 16, 1024)
        upsample(256, 4),  # (bs, 32, 32, 512)
        upsample(128, 4),  # (bs, 64, 64, 256)
        upsample(64, 4),   # (bs, 128, 128, 128)
    ]
    
    last = layers.Conv2DTranspose(3, 4, strides=2, padding='same', kernel_initializer=tf.random_normal_initializer(0., 0.02), activation='tanh')  # (bs, 256, 256, 3)
    
    x = inputs
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    
    skips = reversed(skips[:-1])
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = layers.Concatenate()([x, skip])
    
    x = last(x)
    return tf.keras.Model(inputs=inputs, outputs=x)

# Discriminator: PatchGAN
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    inp = layers.Input(shape=[256, 256, 3])
    x = downsample(64, 4, False)(inp)  
    x = downsample(128, 4)(x) 
    x = downsample(256, 4)(x)  
    x = layers.ZeroPadding2D()(x)
    x = layers.Conv2D(512, 4, strides=1, kernel_initializer=initializer, use_bias=False)(x)  # (bs, 31, 31, 512)
    x = layers.LayerNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.ZeroPadding2D()(x)
    x = layers.Conv2D(1, 4, strides=1, kernel_initializer=initializer)(x)  # (bs, 30, 30, 1)
    return tf.keras.Model(inputs=inp, outputs=x)

# Instantiate
# Photo to Monet
gen_G = Generator()  
# Monet to Photo
gen_F = Generator()  
# For photos
disc_X = Discriminator() 
# For Monets
disc_Y = Discriminator()
'''

"\nfrom tensorflow.keras import layers\n\ndef downsample(filters, size, apply_norm=True):\n    initializer = tf.random_normal_initializer(0., 0.02)\n    result = tf.keras.Sequential()\n    result.add(layers.Conv2D(filters, size, strides=2, padding='same', kernel_initializer=initializer, use_bias=False))\n    if apply_norm:\n        result.add(layers.LayerNormalization())  \n    result.add(layers.LeakyReLU())\n    return result\n\ndef upsample(filters, size, apply_dropout=False):\n    initializer = tf.random_normal_initializer(0., 0.02)\n    result = tf.keras.Sequential()\n    result.add(layers.Conv2DTranspose(filters, size, strides=2, padding='same', kernel_initializer=initializer, use_bias=False))\n    result.add(layers.LayerNormalization())\n    if apply_dropout:\n        result.add(layers.Dropout(0.5))\n    result.add(layers.ReLU())\n    return result\n\n# Generator: U-Net like\ndef Generator():\n    inputs = layers.Input(shape=[256, 256, 3])\n    \n    down_stack = [\n        downs

**Results and Analysis (35 pts)**

Run hyperparameter tuning, try different architectures for comparison, apply techniques to improve training or performance, and discuss what helped.

Includes results with tables and figures. There is an analysis of why or why not something worked well, troubleshooting, and a hyperparameter optimization procedure summary.

Result will speak for themselves, I will train my model on as much time that I have left on my GPU quota. Should get 50 epochs out of it, and then I will save the set of photos and submit them to be scored by kaggles MiFID scoreing system. 

In [27]:
'''
LAMBDA = 10 

loss_obj = tf.keras.losses.MeanSquaredError() 

def discriminator_loss(real, generated):
    real_loss = loss_obj(tf.ones_like(real), real)
    generated_loss = loss_obj(tf.zeros_like(generated), generated)
    return (real_loss + generated_loss) * 0.5

def generator_loss(generated):
    return loss_obj(tf.ones_like(generated), generated)

def calc_cycle_loss(real_image, cycled_image):
    return tf.reduce_mean(tf.abs(real_image - cycled_image)) * LAMBDA

def identity_loss(real_image, same_image):
    return tf.reduce_mean(tf.abs(real_image - same_image)) * LAMBDA * 0.5 

# Optimizers
gen_G_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
gen_F_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
disc_X_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
disc_Y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

# Training step (use @tf.function for speed)
@tf.function
def train_step(real_x, real_y):
    with tf.GradientTape(persistent=True) as tape:
        # Generator G: x -> y
        fake_y = gen_G(real_x, training=True)
        cycled_x = gen_F(fake_y, training=True)
        
        # Generator F: y -> x
        fake_x = gen_F(real_y, training=True)
        cycled_y = gen_G(fake_x, training=True)
        
        # Identity
        same_x = gen_F(real_x, training=True)
        same_y = gen_G(real_y, training=True)
        
        # Discriminators
        disc_real_x = disc_X(real_x, training=True)
        disc_real_y = disc_Y(real_y, training=True)
        disc_fake_x = disc_X(fake_x, training=True)
        disc_fake_y = disc_Y(fake_y, training=True)
        
        # Losses
        gen_G_loss = generator_loss(disc_fake_y) + calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y) + identity_loss(real_y, same_y)
        gen_F_loss = generator_loss(disc_fake_x) + calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y) + identity_loss(real_x, same_x)
        total_gen_loss = gen_G_loss + gen_F_loss
        
        disc_X_loss = discriminator_loss(disc_real_x, disc_fake_x)
        disc_Y_loss = discriminator_loss(disc_real_y, disc_fake_y)
    
    # Gradients
    gen_G_grads = tape.gradient(gen_G_loss, gen_G.trainable_variables)
    gen_F_grads = tape.gradient(gen_F_loss, gen_F.trainable_variables)
    disc_X_grads = tape.gradient(disc_X_loss, disc_X.trainable_variables)
    disc_Y_grads = tape.gradient(disc_Y_loss, disc_Y.trainable_variables)
    
    # Apply
    gen_G_optimizer.apply_gradients(zip(gen_G_grads, gen_G.trainable_variables))
    gen_F_optimizer.apply_gradients(zip(gen_F_grads, gen_F.trainable_variables))
    disc_X_optimizer.apply_gradients(zip(disc_X_grads, disc_X.trainable_variables))
    disc_Y_optimizer.apply_gradients(zip(disc_Y_grads, disc_Y.trainable_variables))
    
    return total_gen_loss, disc_X_loss + disc_Y_loss

# Training loop
EPOCHS = 50 
for epoch in range(EPOCHS):
    for real_monet, real_photo in train_ds:  # Note: Adjust if zipped differently
        gen_loss, disc_loss = train_step(real_photo, real_monet)  # Photo as x, Monet as y
    print(f"Epoch {epoch+1}: Gen loss {gen_loss}, Disc loss {disc_loss}")
    
    # Save checkpoints every 10 epochs (use tf.train.Checkpoint)
    if (epoch + 1) % 10 == 0:
        # TODO: Implement checkpointing
        pass
    
    # Generate sample images for monitoring
    sample_photo = next(iter(photo_ds.take(1)))
    fake_monet = gen_G(sample_photo)
    plt.imshow((fake_monet[0] + 1) / 2)
    plt.show()
    '''

'\nLAMBDA = 10 \n\nloss_obj = tf.keras.losses.MeanSquaredError() \n\ndef discriminator_loss(real, generated):\n    real_loss = loss_obj(tf.ones_like(real), real)\n    generated_loss = loss_obj(tf.zeros_like(generated), generated)\n    return (real_loss + generated_loss) * 0.5\n\ndef generator_loss(generated):\n    return loss_obj(tf.ones_like(generated), generated)\n\ndef calc_cycle_loss(real_image, cycled_image):\n    return tf.reduce_mean(tf.abs(real_image - cycled_image)) * LAMBDA\n\ndef identity_loss(real_image, same_image):\n    return tf.reduce_mean(tf.abs(real_image - same_image)) * LAMBDA * 0.5 \n\n# Optimizers\ngen_G_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)\ngen_F_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)\ndisc_X_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)\ndisc_Y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)\n\n# Training step (use @tf.function for speed)\n@tf.function\ndef train_step(real_x, real_y):\n    with tf.GradientTape(pe

In [28]:
'''from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from PIL import Image
import os
import glob
import zipfile

def load_image(file_path):
    img = load_img(file_path, target_size=(256, 256))
    img = img_to_array(img)
    img = (img / 127.5) - 1.0  # Normalize to [-1, 1] for GAN input
    return img

# Now your code
os.makedirs('images', exist_ok=True)
for i, photo_path in enumerate(photo_files):
    photo = load_image(photo_path)
    photo = np.expand_dims(photo, 0)  # Add batch dimension
    generated = gen_G(photo)[0]  # Generate
    generated = (generated * 127.5 + 127.5).numpy().astype(np.uint8)  # Denormalize to [0, 255]
    img = Image.fromarray(generated)
    img.save(f'images/{i+1:05d}.jpg')

with zipfile.ZipFile('images.zip', 'w') as z:
    for f in glob.glob('images/*.jpg'):
        z.write(f)'''

"from tensorflow.keras.preprocessing.image import load_img, img_to_array\nimport numpy as np\nfrom PIL import Image\nimport os\nimport glob\nimport zipfile\n\ndef load_image(file_path):\n    img = load_img(file_path, target_size=(256, 256))\n    img = img_to_array(img)\n    img = (img / 127.5) - 1.0  # Normalize to [-1, 1] for GAN input\n    return img\n\n# Now your code\nos.makedirs('images', exist_ok=True)\nfor i, photo_path in enumerate(photo_files):\n    photo = load_image(photo_path)\n    photo = np.expand_dims(photo, 0)  # Add batch dimension\n    generated = gen_G(photo)[0]  # Generate\n    generated = (generated * 127.5 + 127.5).numpy().astype(np.uint8)  # Denormalize to [0, 255]\n    img = Image.fromarray(generated)\n    img.save(f'images/{i+1:05d}.jpg')\n\nwith zipfile.ZipFile('images.zip', 'w') as z:\n    for f in glob.glob('images/*.jpg'):\n        z.write(f)"

**Conclusion (15 pts)**

Discuss and interpret results as well as learnings and takeaways. What did and did not help improve the performance of your models? What improvements could you try in the future?